# Part 1.

The deadline for Part 1 is **2 pm Feb 11, 2021**.   
You should submit a `.ipynb` file with your solutions to NYU Classes.

---
Spam filtering is a well-studied NLP classification problem that is used in many commercial products nowadays, including email clients and mobile SMS apps.

In this assignment we will train a logistic regression model to classify each text in the SMS Spam Collection dataset as either spam or legitimate. This dataset consists of 5,574 English SMS messages, each tagged as either spam or ham (legitimate). The column 'v1' contains the label and the column 'v2' contains the SMS text. We will pre-process and convert the texts into bag-of-words features for our model.

## Data Loading

First, we download the SMS Spam Collection Dataset. The below command downloads the dataset from [Kaggle](https://www.kaggle.com/uciml/sms-spam-collection-dataset/data#) and loads it to [Google Drive](https://drive.google.com/open?id=1OVRo37agn02mc6yp5p6-wtJ8Hyb-YMXR).

In [ ]:
!wget 'https://docs.google.com/uc?export=download&id=1OVRo37agn02mc6yp5p6-wtJ8Hyb-YMXR' -O spam.csv 

--2021-02-03 19:46:53--  https://docs.google.com/uc?export=download&id=1OVRo37agn02mc6yp5p6-wtJ8Hyb-YMXR
Resolving docs.google.com (docs.google.com)... 173.194.214.100, 173.194.214.113, 173.194.214.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.214.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-04-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lchte0302sib3tki49t8pf47pr3ck0c3/1612381575000/08752484438609855375/*/1OVRo37agn02mc6yp5p6-wtJ8Hyb-YMXR?e=download [following]
--2021-02-03 19:46:53--  https://doc-14-04-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lchte0302sib3tki49t8pf47pr3ck0c3/1612381575000/08752484438609855375/*/1OVRo37agn02mc6yp5p6-wtJ8Hyb-YMXR?e=download
Resolving doc-14-04-docs.googleusercontent.com (doc-14-04-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to doc-14-04-docs.googleusercontent.com (doc-14

In [ ]:
!ls

sample_data  spam.csv


Now we preview the data. There are two columns: `v1` -- the label, which indicates whether the text is spam or ham (legitimate), and `v2` -- the text of the message.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("spam.csv", usecols=["v1", "v2"], encoding='latin-1')
# 1 - spam, 0 - ham
df.v1 = (df.v1 == "spam").astype("int")
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


Your task is to randomly split the data into training, validation, and test sets. Make sure that each row appears in only one of the splits and that training contains 70% of the data, validation contains 15%, and test contains 15%. You can compute the size of each split afterwards to sanity check your code. **You may use numpy and pandas functions, but please do not use sklearn.**

In [ ]:
# 0.15 for val, 0.15 for test, 0.7 for train
val_size = int(df.shape[0] * 0.15)
test_size = int(df.shape[0] * 0.15)

"""
YOUR CODE GOES HERE
"""

train_texts, train_labels = None, None
val_texts, val_labels     = None, None
test_texts, test_labels   = None, None

## Data Processing

The task is to create bag-of-words features: tokenize the text, index each token, represent the sentence as a dictionary of tokens and their counts, limit the vocabulary to $n$ most frequent tokens. In Lab 2 we will use the built-in `sklearn` function, `sklearn.feature_extraction.text.CountVectorizer`. 
**In this HW, you are required to implement the class `Vectorizer` on your own without using `sklearn` built-in functions.**

Function `preprocess_data` takes the list of texts and returns list of (lists of tokens). 
You may use [spacy](https://spacy.io/) or [nltk](https://www.nltk.org/) text processing libraries in `preprocess_data` function. 

Class `Vectorizer` is used to vectorize the text and to create a matrix of features.


In [ ]:
def preprocess_data(data):
    # This function should return a list of lists of pre-processed tokens, where
    # each nested list contains the tokens for a single message. To pre-process
    # each message, tokenize the message and lowercase all text.
    """
    YOUR CODE GOES HERE
    """
    preprocessed_data = None
    return preprocessed_data

train_data = preprocess_data(train_texts)
val_data = preprocess_data(val_texts)
test_data = preprocess_data(test_texts)

In [ ]:
train_data

In [ ]:
import numpy as np

class Vectorizer():
    def __init__(self, max_features):
        self.max_features = max_features
        self.vocab_list = None
        self.token_to_index = None

    def fit(self, dataset):
        # Given a list of lists of tokens, create a vocab list, self.vocab_list, 
        # using the most frequent "max_features" tokens. Also create a token 
        # indexer, self.token_to_index, that will return the index of the token 
        # in self.vocab_list.
        """
        YOUR CODE GOES HERE
        """
        pass

    def transform(self, dataset):
        # This function transforms the text dataset (a list of lists of tokens) 
        # into a matrix, "data_matrix," where the entry located at (i, j) is 1 
        # if sample i of the dataset contains token j in the vocab list, and 0 
        # otherwise.
        """
        YOUR CODE GOES HERE
        """
        data_matrix = np.zeros((len(dataset), len(self.vocab_list)))
        
        return data_matrix

In [ ]:
max_features = None # TODO: Replace None with a number
vectorizer = Vectorizer(max_features=max_features)
vectorizer.fit(train_data)
X_train = vectorizer.transform(train_data)
X_val = vectorizer.transform(val_data)
X_test = vectorizer.transform(test_data)

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

vocab = vectorizer.vocab_list


You can add more features to the feature matrix. (Not required, but worth extra credit points.)

In [ ]:
"""
YOUR CODE GOES HERE
"""

'\nYOUR CODE GOES HERE\n'

## Model

We train a logistic regression model on the bag-of-words features and save predictions for the training, validation, and test sets.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Define Logistic Regression model
model = LogisticRegression(random_state=0, solver='liblinear')

# Fit the model to training data
model.fit(X_train, y_train)

# Make prediction using the trained model
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

## Performance of the model

Your task is to report train, val, test accuracies and F1 scores.
**You are required to implement `accuracy_score` and `f1_score` methods without using built-in python functions.**

Your model should achieve at least **0.95** test accuracy and **0.90** test F1 score.

In [ ]:
def accuracy_score(y_true, y_pred): 
    # Calculate accuracy of the model's prediction
    """
    YOUR CODE GOES HERE
    """
    accuracy = None
    return accuracy

def f1_score(y_true, y_pred): 
    # Calculate F1 score of the model's prediction
    """
    YOUR CODE GOES HERE
    """
    f1 = None
    return f1

In [ ]:
print(f"Training accuracy: {accuracy_score(y_train, y_train_pred):.3f}, "
      f"F1 score: {f1_score(y_train, y_train_pred):.3f}")
print(f"Validation accuracy: {accuracy_score(y_val, y_val_pred):.3f}, "
      f"F1 score: {f1_score(y_val, y_val_pred):.3f}")
print(f"Test accuracy: {accuracy_score(y_test, y_test_pred):.3f}, "
      f"F1 score: {f1_score(y_test, y_test_pred):.3f}")

**Question.**
Is accuracy the metric that logistic regression optimizes while training? If no, which metric is optimized in logistic regression?

**Your answer:** 

**Question.**
In general, does having 0.99 accuracy on test means that the model is great? If no, can you give an example of a case when the accuracy is high but the model is not good? (Hint: why do we use F1 score?)

**Your answer:** 

### Exploration of predictions

Show a few examples with correctly predicted labels on the train and val sets.

In [ ]:
"""
YOUR CODE GOES HERE
"""
# 1 - spam, 0 - ham


**Question** Print 10 examples from val set which were labeled incorrectly by the model. Why do you think the model got them wrong?

**Your answer:** 

In [ ]:
"""
YOUR CODE GOES HERE
"""

## End of Part 1.
